In [1]:
import pandas as pd
import numpy as np
import re

In [3]:
br_tsv = pd.read_csv('data/US_BATON_ROUGE_trainingdata.tsv', sep='\t', low_memory=False)
br_csv = pd.read_csv('data/BatonRouge_311_Citizen_Requests_for_Service.csv', low_memory=False)

In [6]:
br_tsv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232537 entries, 0 to 232536
Data columns (total 10 columns):
Unnamed: 0        232537 non-null int64
COMPLAINT ID      232537 non-null object
CITY              232537 non-null object
DEPT_311          232537 non-null object
CODE_311          232537 non-null object
COMPLAINT_1       227137 non-null object
COMPLAINT_2       0 non-null float64
COMPLAINT DATE    232537 non-null object
CATEGORY_MAIN     232537 non-null object
CATEGORY_SUB      232537 non-null object
dtypes: float64(1), int64(1), object(8)
memory usage: 17.7+ MB


In [7]:
br_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232537 entries, 0 to 232536
Data columns (total 23 columns):
SERVICE REQUEST ID          232537 non-null int64
STATUS                      232537 non-null object
CREATE DATE                 232537 non-null object
CLOSE DATE                  208995 non-null object
LAST ACTION DATE            232537 non-null object
PARENT TYPE                 232537 non-null object
TYPE                        232537 non-null object
COMMENTS                    227137 non-null object
STREET NUMBER               200148 non-null object
STREET NAME                 229692 non-null object
FULL ADDRESS                230343 non-null object
CROSS STREET                19348 non-null object
CITY                        232537 non-null object
DEPARTMENT                  232537 non-null object
DIVISION                    232537 non-null object
LATITUDE                    232537 non-null float64
LONGITUDE                   232537 non-null float64
GEOLOCATION           

In [8]:
br_tsv.head()

,Unnamed: 0,COMPLAINT ID,CITY,DEPT_311,CODE_311,COMPLAINT_1,COMPLAINT_2,COMPLAINT DATE,CATEGORY_MAIN,CATEGORY_SUB
0,0,US_BATON_ROUGE_220943,US_BATON_ROUGE,ROAD MAINTENANCE ISSUES,POTHOLE,POTHOLE ON THE SW CORNER OF HILMONT AND SPRING...,NaN,03/22/2016 04:40:00 PM,STREET,STREET_REPAIR
1,1,US_BATON_ROUGE_248677,US_BATON_ROUGE,STREET/TRAFFIC ISSUES,ROAD STRIPING NEEDED,STREET NEEDS TO BE RE STRIPED AT THE CORNER SO...,NaN,07/27/2016 02:45:00 PM,STREET,STREET_REPAIR
2,2,US_BATON_ROUGE_251335,US_BATON_ROUGE,"DRAINAGE, EROSION, FLOODING OR HOLES",CAVE-IN/SINK HOLE (DRAINAGE RELATED),"STORM DRAINS N FRONTOF HOMR ARE CAVING IN, LI...",NaN,08/09/2016 07:48:00 AM,INFRASTRUCTURE,INFRASTRUCTURE_GENERAL
3,3,US_BATON_ROUGE_286906,US_BATON_ROUGE,BLIGHTED PROPERTIES,PROPERTY MISSING WINDOWS OR DOORS,VACANT PROPERTY AT 2230 MARYLAND STREET IS UNS...,NaN,01/03/2017 01:49:00 PM,HOUSING,HOUSING_GENERAL
4,4,US_BATON_ROUGE_293409,US_BATON_ROUGE,ROAD MAINTENANCE ISSUES,POTHOLE,ROAD IS SINKING ROAD SUB BASE NEED TO BE REPAI...,NaN,02/01/2017 04:41:00 PM,STREET,STREET_REPAIR


In [9]:
br_csv.head()

,SERVICE REQUEST ID,STATUS,CREATE DATE,CLOSE DATE,LAST ACTION DATE,PARENT TYPE,TYPE,COMMENTS,STREET NUMBER,STREET NAME,...,DEPARTMENT,DIVISION,LATITUDE,LONGITUDE,GEOLOCATION,TYPEI D,PARENT ID,DEPTDIV ID,DEPARTMENT-DIVISION NAME,STATUS ID
0,220943,CLOSED,03/22/2016 04:40:00 PM,07/06/2018 02:09:00 PM,07/06/2018 02:09:00 PM,ROAD MAINTENANCE ISSUES,POTHOLE,POTHOLE ON THE SW CORNER OF HILMONT AND SPRING...,10244,SPRINGLAKE DR,...,MAINTENANCE,STREET MAINTENANCE,30.342981,-91.114136,"(30.342981, -91.114136)",173071,172980,64,DOM - STREET MAINTENANCE,1
1,248677,CLOSED,07/27/2016 02:45:00 PM,03/12/2018 11:24:00 AM,03/12/2018 11:24:00 AM,STREET/TRAFFIC ISSUES,ROAD STRIPING NEEDED,STREET NEEDS TO BE RE STRIPED AT THE CORNER SO...,NaN,SOUTH BLVD,...,TRANSPORTATION AND DEVELOPMENT,TRAFFIC ENGINEERING,30.439724,-91.174988,"(30.439724, -91.174988)",173102,172982,66,TAD - TRAFFIC ENGINEERING,1
2,251335,CLOSED,08/09/2016 07:48:00 AM,03/12/2018 10:19:00 AM,03/12/2018 10:19:00 AM,"DRAINAGE, EROSION, FLOODING OR HOLES",CAVE-IN/SINK HOLE (DRAINAGE RELATED),"STORM DRAINS N FRONTOF HOMR ARE CAVING IN, LI...",1916,E BELFAIR DR,...,MAINTENANCE,DRAINAGE MAINTENANCE,30.465631,-91.146400,"(30.465631, -91.1464)",172998,172973,62,DOM - DRAINAGE MAINTENANCE,1
3,286906,IN PROGRESS,01/03/2017 01:49:00 PM,NaN,07/06/2018 10:48:00 AM,BLIGHTED PROPERTIES,PROPERTY MISSING WINDOWS OR DOORS,VACANT PROPERTY AT 2230 MARYLAND STREET IS UNS...,2230,MARYLAND ST,...,DEVELOPMENT,BLIGHT ENFORCEMENT,30.426577,-91.173882,"(30.426577, -91.173882)",172987,172971,59,DOD - BLIGHT ENFORCEMENT,3
4,293409,CLOSED,02/01/2017 04:41:00 PM,07/06/2018 02:08:00 PM,07/06/2018 02:08:00 PM,ROAD MAINTENANCE ISSUES,POTHOLE,ROAD IS SINKING ROAD SUB BASE NEED TO BE REPAI...,NaN,PENTAGON CT,...,MAINTENANCE,STREET MAINTENANCE,30.321110,-91.123695,"(30.32111, -91.123695)",173071,172980,64,DOM - STREET MAINTENANCE,1
